# Chapter 16: Authentication & Authorization

Authentication and authorization are critical components of modern web applications. Next.js 13+ with the App Router introduces new patterns for handling authentication, integrating seamlessly with Server Components, Server Actions, and Middleware. Whether you're implementing social login, credential-based auth, or complex role-based access control, understanding these patterns is essential for building secure applications.

By the end of this chapter, you'll master authentication fundamentals, integrate NextAuth.js (Auth.js) with the App Router, configure OAuth providers, implement credential authentication, manage sessions effectively, protect routes with middleware, and implement role-based access control with security best practices.

## 16.1 Authentication Fundamentals

Understanding the authentication flow in Next.js helps you choose the right strategy for your application.

### Authentication Architecture in App Router

The App Router introduces a hybrid authentication model:

```typescript
// Architecture Overview
/*
Server Components:
- Can read sessions directly via cookies/headers
- Can check authentication status without client JS
- Cannot use React Context for auth state

Client Components:
- Handle interactive auth flows (login forms, buttons)
- Manage client-side session state
- Use Context or hooks for auth awareness

Middleware:
- Runs at the edge before requests reach the app
- Validates JWTs/session tokens
- Redirects unauthenticated users
- Adds auth headers to requests
*/
```

### Session Strategies

Choose between JWT (stateless) and Database (stateful) sessions:

```typescript
// lib/auth/config.ts
import { NextAuthOptions } from 'next-auth';

export const authConfig: NextAuthOptions = {
  // JWT Strategy: Good for serverless, scalable, no DB lookups
  session: {
    strategy: 'jwt',
    maxAge: 30 * 24 * 60 * 60, // 30 days
  },
  
  // Database Strategy: Revocable sessions, larger data storage
  // session: {
  //   strategy: 'database',
  //   maxAge: 30 * 24 * 60 * 60,
  //   updateAge: 24 * 60 * 60, // Update session every 24h
  // },
  
  // Database adapter required for database strategy
  // adapter: PrismaAdapter(prisma),
};
```

### Token Management

Handle token refresh and validation:

```typescript
// lib/auth/tokens.ts
import { jwtVerify, SignJWT } from 'jose';
import { cookies } from 'next/headers';

const secret = new TextEncoder().encode(process.env.JWT_SECRET);

export async function createToken(payload: any, expiresIn: string = '7d') {
  return new SignJWT(payload)
    .setProtectedHeader({ alg: 'HS256' })
    .setIssuedAt()
    .setExpirationTime(expiresIn)
    .sign(secret);
}

export async function verifyToken(token: string) {
  try {
    const { payload } = await jwtVerify(token, secret);
    return payload;
  } catch (error) {
    return null;
  }
}

// Server Component usage
export async function getSession() {
  const cookieStore = cookies();
  const token = cookieStore.get('auth-token')?.value;
  
  if (!token) return null;
  return verifyToken(token);
}
```

## 16.2 NextAuth.js Integration

NextAuth.js (now Auth.js) is the most popular authentication library for Next.js, supporting OAuth, email, and credentials providers.

### Basic Setup

Configure NextAuth.js with the App Router:

```typescript
// app/api/auth/[...nextauth]/route.ts
import NextAuth from 'next-auth';
import { authOptions } from '@/lib/auth/options';

const handler = NextAuth(authOptions);
export { handler as GET, handler as POST };
```

```typescript
// lib/auth/options.ts
import { NextAuthOptions } from 'next-auth';
import GoogleProvider from 'next-auth/providers/google';
import GitHubProvider from 'next-auth/providers/github';
import { PrismaAdapter } from '@auth/prisma-adapter';
import { prisma } from '@/lib/db';

export const authOptions: NextAuthOptions = {
  adapter: PrismaAdapter(prisma),
  providers: [
    GoogleProvider({
      clientId: process.env.GOOGLE_CLIENT_ID!,
      clientSecret: process.env.GOOGLE_CLIENT_SECRET!,
    }),
    GitHubProvider({
      clientId: process.env.GITHUB_ID!,
      clientSecret: process.env.GITHUB_SECRET!,
    }),
  ],
  session: {
    strategy: 'database', // or 'jwt'
  },
  pages: {
    signIn: '/auth/signin',
    signOut: '/auth/signout',
    error: '/auth/error',
  },
  callbacks: {
    async session({ session, user, token }) {
      // Add custom data to session
      if (session.user) {
        session.user.id = user?.id || token?.sub;
        session.user.role = user?.role || 'user';
      }
      return session;
    },
    async jwt({ token, user, account }) {
      // Persist OAuth access_token to token right after signin
      if (account) {
        token.accessToken = account.access_token;
      }
      if (user) {
        token.role = user.role;
      }
      return token;
    },
  },
  events: {
    async signIn({ user, account, profile }) {
      console.log(`User ${user.email} signed in`);
    },
  },
};
```

### TypeScript Augmentation

Extend NextAuth types for custom fields:

```typescript
// types/next-auth.d.ts
import NextAuth from 'next-auth';

declare module 'next-auth' {
  interface Session {
    user: {
      id: string;
      role: 'user' | 'admin' | 'editor';
      email: string;
      name: string;
      image?: string;
    };
  }

  interface User {
    role?: 'user' | 'admin' | 'editor';
  }
}

declare module 'next-auth/jwt' {
  interface JWT {
    role?: 'user' | 'admin' | 'editor';
    accessToken?: string;
  }
}
```

### Session Provider Setup

Wrap your app with the SessionProvider for client-side session access:

```typescript
// components/providers/session-provider.tsx
'use client';

import { SessionProvider as Provider } from 'next-auth/react';
import { Session } from 'next-auth';

export function SessionProvider({ 
  children, 
  session 
}: { 
  children: React.ReactNode;
  session: Session | null;
}) {
  return <Provider session={session}>{children}</Provider>;
}

// app/layout.tsx
import { getServerSession } from 'next-auth';
import { SessionProvider } from '@/components/providers/session-provider';
import { authOptions } from '@/lib/auth/options';

export default async function RootLayout({ 
  children 
}: { 
  children: React.ReactNode;
}) {
  const session = await getServerSession(authOptions);
  
  return (
    <html lang="en">
      <body>
        <SessionProvider session={session}>
          {children}
        </SessionProvider>
      </body>
    </html>
  );
}
```

## 16.3 OAuth Providers

Configure popular OAuth providers for seamless social login.

### Google Provider Setup

Configure Google OAuth 2.0:

```typescript
// lib/auth/providers.ts
import GoogleProvider from 'next-auth/providers/google';

export const googleProvider = GoogleProvider({
  clientId: process.env.GOOGLE_CLIENT_ID!,
  clientSecret: process.env.GOOGLE_CLIENT_SECRET!,
  authorization: {
    params: {
      prompt: 'consent',
      access_type: 'offline',
      response_type: 'code',
      scope: [
        'openid',
        'email',
        'profile',
        'https://www.googleapis.com/auth/calendar.readonly', // Additional scopes
      ].join(' '),
    },
  },
  profile(profile) {
    return {
      id: profile.sub,
      name: profile.name,
      email: profile.email,
      image: profile.picture,
      role: 'user', // Default role
    };
  },
});
```

### GitHub Provider

GitHub OAuth configuration:

```typescript
import GitHubProvider from 'next-auth/providers/github';

export const githubProvider = GitHubProvider({
  clientId: process.env.GITHUB_ID!,
  clientSecret: process.env.GITHUB_SECRET!,
  authorization: {
    params: {
      scope: 'read:user user:email',
    },
  },
  profile(profile) {
    return {
      id: profile.id.toString(),
      name: profile.name || profile.login,
      email: profile.email,
      image: profile.avatar_url,
      role: profile.type === 'User' ? 'user' : 'organization',
    };
  },
});
```

### Custom OAuth Provider

Configure any OAuth 2.0 provider:

```typescript
import { OAuthConfig } from 'next-auth/providers';

export const customProvider: OAuthConfig<any> = {
  id: 'custom-provider',
  name: 'CustomProvider',
  type: 'oauth',
  wellKnown: 'https://custom-provider.com/.well-known/openid-configuration',
  authorization: { params: { scope: 'openid email profile' } },
  clientId: process.env.CUSTOM_ID!,
  clientSecret: process.env.CUSTOM_SECRET!,
  checks: ['pkce', 'state'],
  profile(profile) {
    return {
      id: profile.sub,
      name: profile.name,
      email: profile.email,
      image: profile.picture,
    };
  },
};
```

### OAuth Callback Handling

Handle OAuth callbacks for additional processing:

```typescript
// lib/auth/options.ts
export const authOptions: NextAuthOptions = {
  providers: [/* ... */],
  callbacks: {
    async signIn({ user, account, profile, email, credentials }) {
      // Allow only specific domains
      if (user.email && !user.email.endsWith('@company.com')) {
        return false; // Deny sign-in
      }
      
      // Check if user exists in database
      const existingUser = await prisma.user.findUnique({
        where: { email: user.email! },
      });
      
      if (!existingUser) {
        // Auto-create user for OAuth sign-ins
        await prisma.user.create({
          data: {
            email: user.email!,
            name: user.name,
            image: user.image,
            role: 'user',
          },
        });
      }
      
      return true;
    },
    
    async redirect({ url, baseUrl }) {
      // Redirect to dashboard after sign-in
      if (url.startsWith(baseUrl)) {
        return `${baseUrl}/dashboard`;
      }
      return baseUrl;
    },
  },
};
```

## 16.4 Credential Authentication

Implement email/password authentication with secure password handling.

### Credentials Provider

Set up username/password login:

```typescript
// lib/auth/options.ts
import CredentialsProvider from 'next-auth/providers/credentials';
import bcrypt from 'bcryptjs';

export const authOptions: NextAuthOptions = {
  providers: [
    CredentialsProvider({
      name: 'credentials',
      credentials: {
        email: { label: 'Email', type: 'email' },
        password: { label: 'Password', type: 'password' },
      },
      async authorize(credentials) {
        if (!credentials?.email || !credentials?.password) {
          return null;
        }
        
        // Find user
        const user = await prisma.user.findUnique({
          where: { email: credentials.email },
        });
        
        if (!user || !user.password) {
          return null; // User not found or OAuth-only account
        }
        
        // Verify password
        const isValid = await bcrypt.compare(
          credentials.password,
          user.password
        );
        
        if (!isValid) {
          return null;
        }
        
        // Update last login
        await prisma.user.update({
          where: { id: user.id },
          data: { lastLogin: new Date() },
        });
        
        return {
          id: user.id,
          email: user.email,
          name: user.name,
          role: user.role,
        };
      },
    }),
  ],
};
```

### Registration with Server Actions

Handle user registration securely:

```typescript
// app/auth/register/actions.ts
'use server';

import bcrypt from 'bcryptjs';
import { prisma } from '@/lib/db';
import { z } from 'zod';

const registerSchema = z.object({
  name: z.string().min(2),
  email: z.string().email(),
  password: z.string().min(8),
});

export async function register(formData: FormData) {
  const data = Object.fromEntries(formData);
  const parsed = registerSchema.safeParse(data);
  
  if (!parsed.success) {
    return { error: 'Invalid data', details: parsed.error.errors };
  }
  
  const { name, email, password } = parsed.data;
  
  // Check existing user
  const existing = await prisma.user.findUnique({
    where: { email },
  });
  
  if (existing) {
    return { error: 'Email already registered' };
  }
  
  // Hash password
  const hashedPassword = await bcrypt.hash(password, 12);
  
  // Create user
  const user = await prisma.user.create({
    data: {
      name,
      email,
      password: hashedPassword,
      role: 'user',
    },
  });
  
  return { success: true, userId: user.id };
}
```

```typescript
// app/auth/register/page.tsx
'use client';

import { useState } from 'react';
import { register } from './actions';

export default function RegisterPage() {
  const [error, setError] = useState('');
  
  async function handleSubmit(formData: FormData) {
    const result = await register(formData);
    if (result.error) {
      setError(result.error);
    } else {
      // Redirect to login
      window.location.href = '/auth/signin';
    }
  }
  
  return (
    <form action={handleSubmit} className="space-y-4 max-w-md mx-auto">
      <h1 className="text-2xl font-bold">Create Account</h1>
      
      {error && (
        <div className="bg-red-100 text-red-700 p-3 rounded">{error}</div>
      )}
      
      <div>
        <label className="block text-sm font-medium">Name</label>
        <input 
          name="name" 
          required 
          className="w-full border rounded p-2"
        />
      </div>
      
      <div>
        <label className="block text-sm font-medium">Email</label>
        <input 
          name="email" 
          type="email" 
          required 
          className="w-full border rounded p-2"
        />
      </div>
      
      <div>
        <label className="block text-sm font-medium">Password</label>
        <input 
          name="password" 
          type="password" 
          required 
          minLength={8}
          className="w-full border rounded p-2"
        />
      </div>
      
      <button 
        type="submit"
        className="w-full bg-blue-500 text-white py-2 rounded"
      >
        Register
      </button>
    </form>
  );
}
```

## 16.5 Session Management

Handle sessions securely with proper validation and refresh strategies.

### Server Component Session Access

Access session data in Server Components:

```typescript
// lib/auth/session.ts
import { getServerSession } from 'next-auth';
import { authOptions } from './options';
import { cache } from 'react';

// Cache session to avoid repeated calls in same request
export const getCachedSession = cache(async () => {
  return getServerSession(authOptions);
});

// app/dashboard/page.tsx
import { getCachedSession } from '@/lib/auth/session';
import { redirect } from 'next/navigation';

export default async function DashboardPage() {
  const session = await getCachedSession();
  
  if (!session) {
    redirect('/auth/signin');
  }
  
  return (
    <div>
      <h1>Welcome, {session.user.name}</h1>
      <p>Role: {session.user.role}</p>
    </div>
  );
}
```

### Client Component Session Hook

Use the useSession hook in Client Components:

```typescript
// components/user-nav.tsx
'use client';

import { useSession, signOut } from 'next-auth/react';
import Link from 'next/link';

export function UserNav() {
  const { data: session, status } = useSession();
  
  if (status === 'loading') {
    return <div>Loading...</div>;
  }
  
  if (!session) {
    return (
      <Link href="/auth/signin" className="text-blue-500">
        Sign In
      </Link>
    );
  }
  
  return (
    <div className="flex items-center gap-4">
      <span>{session.user.name}</span>
      <button 
        onClick={() => signOut({ callbackUrl: '/' })}
        className="text-red-500"
      >
        Sign Out
      </button>
    </div>
  );
}
```

### Session Refresh Strategy

Implement token refresh for OAuth providers:

```typescript
// lib/auth/options.ts
export const authOptions: NextAuthOptions = {
  providers: [/* ... */],
  callbacks: {
    async jwt({ token, user, account }) {
      // Initial sign in
      if (account && user) {
        return {
          accessToken: account.access_token,
          refreshToken: account.refresh_token,
          accessTokenExpires: Date.now() + (account.expires_in || 3600) * 1000,
          user,
        };
      }
      
      // Return previous token if not expired
      if (Date.now() < (token.accessTokenExpires as number)) {
        return token;
      }
      
      // Access token expired, refresh it
      return refreshAccessToken(token);
    },
  },
};

async function refreshAccessToken(token: any) {
  try {
    const response = await fetch('https://oauth2.googleapis.com/token', {
      method: 'POST',
      headers: { 'Content-Type': 'application/x-www-form-urlencoded' },
      body: new URLSearchParams({
        client_id: process.env.GOOGLE_CLIENT_ID!,
        client_secret: process.env.GOOGLE_CLIENT_SECRET!,
        grant_type: 'refresh_token',
        refresh_token: token.refreshToken,
      }),
    });
    
    const refreshedTokens = await response.json();
    
    if (!response.ok) {
      throw refreshedTokens;
    }
    
    return {
      ...token,
      accessToken: refreshedTokens.access_token,
      accessTokenExpires: Date.now() + refreshedTokens.expires_in * 1000,
      refreshToken: refreshedTokens.refresh_token ?? token.refreshToken,
    };
  } catch (error) {
    return { ...token, error: 'RefreshAccessTokenError' };
  }
}
```

## 16.6 Protected Routes

Implement route protection using Middleware and component-level checks.

### Middleware Protection

Protect routes at the edge:

```typescript
// middleware.ts
import { withAuth } from 'next-auth/middleware';
import { NextResponse } from 'next/server';

export default withAuth(
  function middleware(req) {
    const token = req.nextauth.token;
    const path = req.nextUrl.pathname;
    
    // Check role-based access
    if (path.startsWith('/admin') && token?.role !== 'admin') {
      return NextResponse.rewrite(new URL('/unauthorized', req.url));
    }
    
    if (path.startsWith('/api/admin') && token?.role !== 'admin') {
      return NextResponse.json({ error: 'Forbidden' }, { status: 403 });
    }
    
    return NextResponse.next();
  },
  {
    callbacks: {
      authorized({ req, token }) {
        if (!token) return false;
        
        // Allow public API routes
        if (req.nextUrl.pathname.startsWith('/api/public')) {
          return true;
        }
        
        return true;
      },
    },
    pages: {
      signIn: '/auth/signin',
    },
  }
);

export const config = {
  matcher: ['/dashboard/:path*', '/admin/:path*', '/api/protected/:path*'],
};
```

### Component-Level Guards

Protect specific components:

```typescript
// components/protected.tsx
'use client';

import { useSession } from 'next-auth/react';
import { useRouter } from 'next/navigation';
import { useEffect } from 'react';

export function Protected({ 
  children,
  requiredRole,
}: { 
  children: React.ReactNode;
  requiredRole?: string;
}) {
  const { data: session, status } = useSession();
  const router = useRouter();
  
  useEffect(() => {
    if (status === 'unauthenticated') {
      router.push('/auth/signin');
    }
    
    if (requiredRole && session?.user?.role !== requiredRole) {
      router.push('/unauthorized');
    }
  }, [status, session, requiredRole, router]);
  
  if (status === 'loading') {
    return <div>Loading...</div>;
  }
  
  if (status === 'unauthenticated') {
    return null;
  }
  
  if (requiredRole && session?.user?.role !== requiredRole) {
    return null;
  }
  
  return <>{children}</>;
}
```

### API Route Protection

Secure API routes with authentication:

```typescript
// app/api/user/profile/route.ts
import { getServerSession } from 'next-auth';
import { authOptions } from '@/lib/auth/options';
import { NextResponse } from 'next/server';

export async function GET() {
  const session = await getServerSession(authOptions);
  
  if (!session) {
    return NextResponse.json({ error: 'Unauthorized' }, { status: 401 });
  }
  
  // Fetch user data
  const user = await prisma.user.findUnique({
    where: { id: session.user.id },
    select: { id: true, name: true, email: true, role: true },
  });
  
  return NextResponse.json({ user });
}

export async function PATCH(request: Request) {
  const session = await getServerSession(authOptions);
  
  if (!session) {
    return NextResponse.json({ error: 'Unauthorized' }, { status: 401 });
  }
  
  const body = await request.json();
  
  // Users can only update their own profile (unless admin)
  if (body.id && body.id !== session.user.id && session.user.role !== 'admin') {
    return NextResponse.json({ error: 'Forbidden' }, { status: 403 });
  }
  
  const updated = await prisma.user.update({
    where: { id: session.user.id },
    data: body,
  });
  
  return NextResponse.json({ user: updated });
}
```

## 16.7 Role-Based Access Control

Implement granular permissions based on user roles.

### Database Schema

Define roles in your database:

```typescript
// schema.prisma
model User {
  id            String    @id @default(cuid())
  email         String    @unique
  name          String?
  role          Role      @default(USER)
  createdAt     DateTime  @default(now())
  updatedAt     DateTime  @updatedAt
}

enum Role {
  USER
  EDITOR
  ADMIN
  SUPERADMIN
}
```

### Role Checking Utilities

Create reusable role checking functions:

```typescript
// lib/auth/roles.ts
import { getServerSession } from 'next-auth';
import { authOptions } from './options';
import { redirect } from 'next/navigation';

type Role = 'USER' | 'EDITOR' | 'ADMIN' | 'SUPERADMIN';

const roleHierarchy: Record<Role, number> = {
  USER: 0,
  EDITOR: 1,
  ADMIN: 2,
  SUPERADMIN: 3,
};

export function hasRole(userRole: Role, requiredRole: Role): boolean {
  return roleHierarchy[userRole] >= roleHierarchy[requiredRole];
}

export async function requireRole(requiredRole: Role) {
  const session = await getServerSession(authOptions);
  
  if (!session) {
    redirect('/auth/signin');
  }
  
  const userRole = (session.user.role as Role) || 'USER';
  
  if (!hasRole(userRole, requiredRole)) {
    redirect('/unauthorized');
  }
  
  return session;
}

// Usage in Server Components
export default async function AdminPage() {
  const session = await requireRole('ADMIN');
  
  return <div>Admin Dashboard</div>;
}
```

### Permission-Based Access

Fine-grained permissions beyond roles:

```typescript
// lib/auth/permissions.ts
type Permission = 
  | 'posts:create'
  | 'posts:edit'
  | 'posts:delete'
  | 'users:manage'
  | 'settings:modify';

const rolePermissions: Record<string, Permission[]> = {
  USER: ['posts:create'],
  EDITOR: ['posts:create', 'posts:edit'],
  ADMIN: ['posts:create', 'posts:edit', 'posts:delete', 'users:manage'],
  SUPERADMIN: ['*'], // All permissions
};

export function hasPermission(role: string, permission: Permission): boolean {
  const permissions = rolePermissions[role] || [];
  return permissions.includes('*') || permissions.includes(permission);
}

// Component usage
'use client';

import { useSession } from 'next-auth/react';
import { hasPermission } from '@/lib/auth/permissions';

export function DeletePostButton({ postId }: { postId: string }) {
  const { data: session } = useSession();
  const canDelete = session?.user?.role && 
    hasPermission(session.user.role, 'posts:delete');
  
  if (!canDelete) return null;
  
  return (
    <button 
      onClick={() => deletePost(postId)}
      className="text-red-500"
    >
      Delete
    </button>
  );
}
```

## 16.8 Security Best Practices

Secure your authentication implementation against common vulnerabilities.

### CSRF Protection

NextAuth.js handles CSRF automatically, but be aware for custom implementations:

```typescript
// lib/csrf.ts
import { cookies } from 'next/headers';

export function validateCSRFToken(formToken: string): boolean {
  const cookieStore = cookies();
  const csrfToken = cookieStore.get('next-auth.csrf-token')?.value;
  
  if (!csrfToken) return false;
  
  // CSRF token is double-submitted (cookie + form)
  const [cookieValue] = csrfToken.split('|');
  return cookieValue === formToken;
}

// In Server Actions
export async function sensitiveAction(formData: FormData) {
  const csrfToken = formData.get('csrfToken') as string;
  
  if (!validateCSRFToken(csrfToken)) {
    throw new Error('Invalid CSRF token');
  }
  
  // Proceed with action
}
```

### Secure Cookie Configuration

Configure cookies for production security:

```typescript
// lib/auth/options.ts
export const authOptions: NextAuthOptions = {
  cookies: {
    sessionToken: {
      name: `__Secure-next-auth.session-token`,
      options: {
        httpOnly: true,
        sameSite: 'lax',
        path: '/',
        secure: process.env.NODE_ENV === 'production',
        domain: process.env.NODE_ENV === 'production' 
          ? '.yourdomain.com' 
          : undefined,
      },
    },
  },
  // Use database sessions for better security (revocable)
  session: {
    strategy: 'database',
    maxAge: 30 * 24 * 60 * 60, // 30 days
    updateAge: 24 * 60 * 60, // Update every 24h
  },
};
```

### Password Security

Implement strong password policies:

```typescript
// lib/auth/password.ts
import { z } from 'zod';
import bcrypt from 'bcryptjs';

const passwordSchema = z.string()
  .min(8, 'Password must be at least 8 characters')
  .regex(/[A-Z]/, 'Must contain uppercase letter')
  .regex(/[a-z]/, 'Must contain lowercase letter')
  .regex(/[0-9]/, 'Must contain number')
  .regex(/[^A-Za-z0-9]/, 'Must contain special character');

export function validatePassword(password: string): { valid: boolean; errors: string[] } {
  const result = passwordSchema.safeParse(password);
  
  if (result.success) {
    return { valid: true, errors: [] };
  }
  
  return { 
    valid: false, 
    errors: result.error.errors.map(e => e.message) 
  };
}

export async function hashPassword(password: string): Promise<string> {
  return bcrypt.hash(password, 12); // 12 rounds recommended
}

export async function verifyPassword(
  password: string, 
  hashedPassword: string
): Promise<boolean> {
  return bcrypt.compare(password, hashedPassword);
}
```

### Rate Limiting

Protect authentication endpoints:

```typescript
// lib/rate-limit.ts
import { LRUCache } from 'lru-cache';

export function createRateLimiter(
  maxRequests: number,
  windowMs: number
) {
  const cache = new LRUCache<string, number[]>({
    max: 500,
    ttl: windowMs,
  });

  return {
    isLimited: (key: string): boolean => {
      const timestamps = cache.get(key) || [];
      const now = Date.now();
      
      // Remove old timestamps
      const recent = timestamps.filter(t => now - t < windowMs);
      
      if (recent.length >= maxRequests) {
        return true;
      }
      
      recent.push(now);
      cache.set(key, recent);
      return false;
    },
  };
}

// In login action
const loginLimiter = createRateLimiter(5, 15 * 60 * 1000); // 5 attempts per 15min

export async function login(formData: FormData) {
  const ip = headers().get('x-forwarded-for') || 'unknown';
  
  if (loginLimiter.isLimited(ip)) {
    return { error: 'Too many attempts. Try again later.' };
  }
  
  // Proceed with login
}
```

### Security Headers

Add security headers via middleware:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  const response = NextResponse.next();
  
  // Prevent clickjacking
  response.headers.set('X-Frame-Options', 'DENY');
  
  // XSS Protection
  response.headers.set('X-XSS-Protection', '1; mode=block');
  
  // Content Type Options
  response.headers.set('X-Content-Type-Options', 'nosniff');
  
  // Referrer Policy
  response.headers.set('Referrer-Policy', 'strict-origin-when-cross-origin');
  
  // Strict Transport Security
  if (process.env.NODE_ENV === 'production') {
    response.headers.set(
      'Strict-Transport-Security',
      'max-age=31536000; includeSubDomains; preload'
    );
  }
  
  return response;
}
```

## Key Takeaways from Chapter 16

1. **Authentication Architecture**: Next.js App Router supports hybrid authentication—Server Components can read sessions directly via cookies, Client Components handle interactive flows via Context, and Middleware validates requests at the edge. Choose JWT for stateless scalability or Database sessions for revocability.

2. **NextAuth.js Integration**: Configure NextAuth.js in `app/api/auth/[...nextauth]/route.ts` with providers (OAuth, Credentials), callbacks for session/JWT customization, and TypeScript augmentation for type safety. Wrap your layout with `SessionProvider` for client-side session access.

3. **OAuth Providers**: Configure Google, GitHub, or custom OAuth providers with proper scopes and profile mapping. Handle callbacks to restrict domains, auto-create users, and persist additional account data like refresh tokens.

4. **Credential Authentication**: Implement email/password auth using the Credentials provider with bcrypt for password hashing. Handle registration via Server Actions with Zod validation, and never store plaintext passwords.

5. **Session Management**: Use `getServerSession` in Server Components (cached per request) and `useSession` in Client Components. Implement token refresh strategies for OAuth providers to maintain long-lived sessions securely.

6. **Protected Routes**: Protect routes using Middleware with `withAuth` for edge-level validation, component-level guards with `useSession` for UI rendering control, and `getServerSession` in API routes for endpoint security.

7. **Role-Based Access Control**: Implement RBAC with database enums, role hierarchy checking utilities, and fine-grained permissions. Use Server Components for server-side role checks and Middleware for route-level protection.

8. **Security Best Practices**: Enable CSRF protection (built into NextAuth.js), configure secure cookies with `httpOnly`, `secure`, and `sameSite` flags, implement rate limiting on auth endpoints, validate passwords for complexity, and add security headers via Middleware.

## Coming Up Next

**Chapter 17: Database Integration**

Now that your users can authenticate securely, it's time to persist application data. In Chapter 17, we'll explore database selection criteria, ORM integration with Prisma and Drizzle, connection management, database seeding, transactions, query optimization, and migration strategies. You'll learn how to structure your database layer for scalability and performance in Next.js applications.